In [1]:
import pandas as pd

from os import listdir
from os.path import isfile, join

import pymysql

In [2]:

path = '../scripts_scrapping/data_scrapping_csv'
liste_csv = [f for f in listdir(path) if isfile(join(path, f))]

df_total = pd.DataFrame()
for filename in liste_csv:
    
    path = "../scripts_scrapping/data_scrapping_csv/"+filename
    df = pd.read_csv(path)
    df['nom_journal'] = filename[:-4]     
    df_total = df_total.append(df, ignore_index=True)
    
df_total = df_total.drop(['theme'], axis=1)

C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [6]:
df_total

,date,nom_journal,titre
0,2020-12-30,charlie_hebdo,Mairie de Marseille : la cuisine électorale es...
1,2020-12-30,charlie_hebdo,L'État bien profond
2,2020-12-28,charlie_hebdo,Vive la transparence politique
3,2020-12-28,charlie_hebdo,Conseil des imams : le recteur de la Grande Mo...
4,2020-12-23,charlie_hebdo,La charité au programme de La République en ma...
...,...,...,...
1305558,2020-12-31,science_avenir,Les “mégaphones” artisanaux des petits grillons
1305559,2020-12-31,science_avenir,"Covid-19: ""envolée"" des tests la semaine de Noël"
1305560,2020-12-31,science_avenir,"Séismes à Strasbourg : deux ""écarts"" commis à ..."
1305561,2020-12-31,science_avenir,Revue de presse Allemagne : double inhumation ...


In [35]:
connection = pymysql.connect(host='localhost', user='JmAuDiCc', password='mdp', db='journaux_db',charset='UTF8')
cursor=connection.cursor()#cursor == tuple

cursor.execute("select Type,Nom_journal from journaux join type_journaux on journaux.Id_type = type_journaux.Id_type")
for row in cursor:
    print(row)
connection.close()

('actu', 'Le Monde')
('actu', 'Libération')
('actu', 'Le Point')
('actu', 'Le Figaro')
('parodique', 'Gorafi')
('parodique', 'Nord Presse')
('people', 'Closer')
('people', 'Public')
('satirique', 'Charlie Hebdo')
('science', 'Science et Avenir')


In [63]:
df_total.nom_journal.unique()

array(['charlie_hebdo', 'closer', 'figaro', 'gorafi', 'lePoint',
       'le_monde', 'liberation', 'nordpresse', 'public', 'science_avenir'],
      dtype=object)

In [32]:

connection = pymysql.connect(host='localhost', user='JmAuDiCc', password='mdp', db='journaux_db')
cursor=connection.cursor(pymysql.cursors.DictCursor)

#mapping "à la main " id_j/nom_j
cursor.execute("select * from journaux ")
for row in cursor:
    print(row)
connection.close()

{'Id_journal': 1, 'Id_type': 1, 'Nom_journal': 'Le Monde'}
{'Id_journal': 2, 'Id_type': 1, 'Nom_journal': 'Libération'}
{'Id_journal': 3, 'Id_type': 1, 'Nom_journal': 'Le Point'}
{'Id_journal': 4, 'Id_type': 1, 'Nom_journal': 'Le Figaro'}
{'Id_journal': 5, 'Id_type': 2, 'Nom_journal': 'Gorafi'}
{'Id_journal': 6, 'Id_type': 2, 'Nom_journal': 'Nord Presse'}
{'Id_journal': 7, 'Id_type': 3, 'Nom_journal': 'Closer'}
{'Id_journal': 8, 'Id_type': 3, 'Nom_journal': 'Public'}
{'Id_journal': 9, 'Id_type': 4, 'Nom_journal': 'Charlie Hebdo'}
{'Id_journal': 10, 'Id_type': 5, 'Nom_journal': 'Science et Avenir'}


In [10]:
#mapping "à la main " id_j/nom_j
id_dic = {'charlie_hebdo':9, 'closer':7, 'figaro':4, 'gorafi':5, 'lePoint':3,
       'le_monde':1, 'liberation':2, 'nordpresse':6, 'public':8, 'science_avenir':10}

In [11]:
df_total.titre = df_total.titre.astype(str)
#éviter int/float dans les titres

In [84]:
for i,row in df_total[:10].iterrows():
    print(row)

date                                                  2020-12-30
nom_journal                                        charlie_hebdo
titre          Mairie de Marseille : la cuisine électorale es...
Name: 0, dtype: object
date                    2020-12-30
nom_journal          charlie_hebdo
titre          L'État bien profond
Name: 1, dtype: object
date                               2020-12-28
nom_journal                     charlie_hebdo
titre          Vive la transparence politique
Name: 2, dtype: object
date                                                  2020-12-28
nom_journal                                        charlie_hebdo
titre          Conseil des imams : le recteur de la Grande Mo...
Name: 3, dtype: object
date                                                  2020-12-23
nom_journal                                        charlie_hebdo
titre          La charité au programme de La République en ma...
Name: 4, dtype: object
date                                                  202

In [12]:
df_total.replace({"nom_journal": id_dic}, inplace=True)

In [23]:
df_total[:2].values.tolist()

[['2020-12-30',
  9,
  'Mairie de Marseille : la cuisine électorale est dure à avaler'],
 ['2020-12-30', 9, "L'État bien profond"]]

In [33]:
connection = pymysql.connect(host='localhost', user='JmAuDiCc', password='mdp', db='journaux_db')
cursor=connection.cursor(pymysql.cursors.DictCursor)
l_e = []
for i,row in enumerate (df_total.values.tolist()):
    if i % 10000 == 0:
        print(i, i/len(df_total))
    try:    
        cursor.execute("insert into articles (Date_art,Id_journal,Titre) values(%s,%s,%s)", row)
    except Exception as e:
        print(i)
        print(e)
        print(row)
        l_e.append((e,row))
connection.commit()   
connection.close()

0 0.0
10000 0.007659530792462716
20000 0.015319061584925431
30000 0.022978592377388148
40000 0.030638123169850862
50000 0.03829765396231358
60000 0.045957184754776295
70000 0.053616715547239006
80000 0.061276246339701725
90000 0.06893577713216444
100000 0.07659530792462715
110000 0.08425483871708987
120000 0.09191436950955259
130000 0.0995739003020153
140000 0.10723343109447801
150000 0.11489296188694073
160000 0.12255249267940345
170000 0.13021202347186617
180000 0.13787155426432887
190000 0.1455310850567916
200000 0.1531906158492543
210000 0.160850146641717
220000 0.16850967743417974
230000 0.17616920822664245
240000 0.18382873901910518
250000 0.19148826981156788
260000 0.1991478006040306
270000 0.20680733139649332
280000 0.21446686218895603
290000 0.22212639298141876
300000 0.22978592377388146
310000 0.23744545456634417
320000 0.2451049853588069
330000 0.2527645161512696
340000 0.26042404694373233
350000 0.268083577736195
360000 0.27574310852865774
370000 0.2834026393211205
380000 0

In [22]:
1,'a'

(1, 'a')

In [36]:
connection = pymysql.connect(host='localhost', user='JmAuDiCc', password='mdp', db='journaux_db'
                             ,charset='UTF8')
cursor=connection.cursor()#cursor == tuple

cursor.execute("select * from articles join journaux on journaux.Id_journal = articles.Id_journal where articles.Date_art = '2020-12-30' and journaux.Nom_journal = 'Charlie Hebdo' ")
for row in cursor:
    print(row)
connection.close()

(1, 9, 'Mairie de Marseille : la cuisine électorale est dure à avaler', '2020-12-30', 9, 4, 'Charlie Hebdo')
(2, 9, "L'État bien profond", '2020-12-30', 9, 4, 'Charlie Hebdo')
(1911, 9, "Argentine : l'IVG enfin légalisée", '2020-12-30', 9, 4, 'Charlie Hebdo')
(1912, 9, 'Engagez-vous, rengagez-vous dans les troupes coloniales', '2020-12-30', 9, 4, 'Charlie Hebdo')
(2242, 9, 'Saint-Denis : menaces sur le fromager de la République', '2020-12-30', 9, 4, 'Charlie Hebdo')
(4274, 9, "La philanthropie, c'est de la merde", '2020-12-30', 9, 4, 'Charlie Hebdo')
(4730, 9, 'Genet au balcon', '2020-12-30', 9, 4, 'Charlie Hebdo')
(4731, 9, 'Le test Nastasia', '2020-12-30', 9, 4, 'Charlie Hebdo')
